<a href="https://colab.research.google.com/github/Qaisar256/AFR-ConvMixer/blob/main/Grad_Cam_What_is_important_in_Spectrograms%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'brain-spectrograms:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4297782%2F7392775%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T135852Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbc07d6e8ce5ce3c37af006fac7de71c3929639e718ac2a37d37f287e0549d6b6a7ff3c3d7bbd24ac871792d17d21a576b69dbc2679ad1ec621899d7d466cb7cb866e6672241f30ab269f0e5c392a030aab897b8ddba4be1ca9e9c08430ad42589073bde0ddfbc1790f7c97c689d6360a4cbb501100006055f03216a2ff2716e98b433ba593478be0b7c4f09e64a4a5eaeb6187cda7c9fa52f0b04cec93667d83b741c6fea286c1276b72833c63cc7da04987762841a1ea629d8ad244eb7a47acd68d5074acbcaab3b0a26733fedbd3ffdf15fccd411f0332736cff4205074be6b81e483201cdb87382377b7619643d6ad7f452fddb67552af9560805a230b3e9,tf-efficientnet-imagenet-weights:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4304475%2F7402356%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T135852Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Deaae4cf4eaa74da993883cc7c0078821adac15f1ba01dcc74ba46ff2965cb76376f7b3448c2d20fe07162e44eb11248d9c57252ba3dca7c0d6f1dbdb67bbcef6f3bbc6e40d8eac5d2737049a0f8fac09f95dd5c5b9cf86ea1d80891a570abf594c9df995c1815edaaefa7771e50e5e610ec97f74260b6f0ca889aafdc9db0ed6b659e43d6d9f77d0f3fb82376befa8efe77d397f9948381012ab3bd7cd66be85814bb5478da902cf82e84d13343778d56785e578815f14e8338df74fea19f9ab19b32db6ffdf8f9cf1b0c87e450bd032e8094cec889a9282c724d0c91a4907ec1a64dfa018eeab8cee1133eeb92df9f6d0308ab9fd03e3f392c797887fe638b3,brain-eeg-spectrograms:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4334995%2F7447509%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T135852Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Ddae7000227770192d86ca2e949afcf5d9bde431f7bedfdb9c61ce57bf18c289bad47b68628b6e88dd2c877ec7d8abfb1f4aff7ae470d0aad77c54a78a8b0ab78a56a93323a72bf7f64262b405942c154f3a07a3de9aadc04f7ee1f9811eab1dae2ab4ed3e8c2ca33bc264794eb6717291de412b881390788119a84036cde85ead5c18c140f5429d002ccb100205003965bc8bd2d14b56f286807bf9347a19928023d3a33183048e4e2ff99c67d282156cf4bd1237145837b202691f47bf7304a49fdaea48f88c20f84205eadab59d0ef342c5650378f9a8c94a149a2bd92fc0ae99012de8af00aaf54ae092c971dd90c895ad64004d6cba3466cc1189c08b1d3,brain-efficientnet-models-v3-v4-v5:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4336944%2F7450712%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T135852Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dd8cf06a57e7a0e1fd73a4073bf0a4d4d15cde000a7c6e62e33fc02d41774630c1b3bb7ff5248b394d88b085c5ac935994399191cf11d21af4c171b6176d6298580cb45707817122753356cf687d66271c0c6ff223ae5b4a44d69d8d093f583841b389922d03de16a9ddebc643d00693f1e4c9319754b7a843026487f25d378276c7e7e7a210e9deb6551ed35d1758772b4f64cfe7c6e5eba84b260eb05043c6117de7af4bb7854df6c3840f134a61bf13530bc6300087790121393c6958b297b74fe0bbaa461145204cc0518d26881699d6a218d589530c886ead240c450b257959c1ef8909a12caf3eb3b2f327871a81acbc3d42df6f70ccb3be7dd585aeb0e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2861294581 bytes downloaded
Downloaded and uncompressed: brain-spectrograms
[==================================================] 278957218 bytes downloaded
Downloaded and uncompressed: tf-efficientnet-imagenet-weights
[====                                              ] 1540997120 bytes downloaded

# Grad Cam for Kaggle's Brain Comp
This notebook displays Grad Cam for the EfficientNetB0 model trained in version 5 of my EfficientNet starter notebook [here][1]. Grad Cam allows us to see where the model is giving its attention to when it makes a prediction. This helps us understand what is important in the spectrogram images. This knowledge helps us improve preprocessing, or improve model architecture, or engineer better features for our ML models like GBT CatBoost. There is a dicussion about this notebook [here][2]

[1]: https://www.kaggle.com/code/cdeotte/efficientnetb0-starter-lb-0-43
[2]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/472976

# Initialize 2xT4 GPU

In [ ]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import tensorflow as tf
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
print('TensorFlow version =',tf.__version__)

# USE MULTIPLE GPUS
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)<=1:
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f'Using {len(gpus)} GPU')
else:
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} GPUs')

# USE MIXED PRECISION
MIX = True
if MIX:
    tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
    print('Mixed precision enabled')
else:
    print('Using full precision')

###############

VER = 5

# IF THIS EQUALS NONE, THEN WE TRAIN NEW MODELS
# IF THIS EQUALS DISK PATH, THEN WE LOAD PREVIOUSLY TRAINED MODELS
LOAD_MODELS_FROM = '/kaggle/input/brain-efficientnet-models-v3-v4-v5/'

USE_KAGGLE_SPECTROGRAMS = True
USE_EEG_SPECTROGRAMS = True

# Load Train Data and Create Non-Overlapping Eeg Ids

In [ ]:
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))

###########

train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spec_id','min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['max'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

# Read Train Spectrograms and EEG Spectrograms

In [ ]:
READ_SPEC_FILES = False

# READ ALL SPECTROGRAMS
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()

###########

READ_EEG_SPEC_FILES = False

if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for i,e in enumerate(train.eeg_id.values):
        if i%100==0: print(i,', ',end='')
        x = np.load(f'/kaggle/input/brain-eeg-spectrograms/EEG_Spectrograms/{e}.npy')
        all_eegs[e] = x
else:
    all_eegs = np.load('/kaggle/input/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()

print(f'There are {len(all_eegs)} eeg parquets')

# DataLoader

In [ ]:
import albumentations as albu
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
TARS2 = {x:y for y,x in TARS.items()}

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32, shuffle=False, augment=False, mode='train',
                 specs = spectrograms, eeg_specs = all_eegs):

        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mode = mode
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = int( np.ceil( len(self.data) / self.batch_size ) )
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        if self.augment: X = self.__augment_batch(X)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange( len(self.data) )
        if self.shuffle: np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'

        X = np.zeros((len(indexes),128,256,8),dtype='float32')
        y = np.zeros((len(indexes),6),dtype='float32')
        img = np.ones((128,256),dtype='float32')

        for j,i in enumerate(indexes):
            row = self.data.iloc[i]
            if self.mode=='test':
                r = 0
            else:
                r = int( (row['min'] + row['max'])//4 )

            for k in range(4):
                # EXTRACT 300 ROWS OF SPECTROGRAM
                img = self.specs[row.spec_id][r:r+300,k*100:(k+1)*100].T

                # LOG TRANSFORM SPECTROGRAM
                img = np.clip(img,np.exp(-4),np.exp(8))
                img = np.log(img)

                # STANDARDIZE PER IMAGE
                ep = 1e-6
                m = np.nanmean(img.flatten())
                s = np.nanstd(img.flatten())
                img = (img-m)/(s+ep)
                img = np.nan_to_num(img, nan=0.0)

                # CROP TO 256 TIME STEPS
                X[j,14:-14,:,k] = img[:,22:-22] / 2.0

            # EEG SPECTROGRAMS
            img = self.eeg_specs[row.eeg_id]
            X[j,:,:,4:] = img

            if self.mode!='test':
                y[j,] = row[TARGETS]

        return X,y

    def __random_transform(self, img):
        composition = albu.Compose([
            albu.HorizontalFlip(p=0.5),
            #albu.CoarseDropout(max_holes=8,max_height=32,max_width=32,fill_value=0,p=0.5),
        ])
        return composition(image=img)['image']

    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i, ] = self.__random_transform(img_batch[i, ])
        return img_batch

# Build Grad Cam Model

In [ ]:
!pip install --no-index --find-links=/kaggle/input/tf-efficientnet-whl-files /kaggle/input/tf-efficientnet-whl-files/efficientnet-1.1.1-py3-none-any.whl

In [ ]:
import efficientnet.tfkeras as efn

def build_cam_model(pretrain=None):

    inp = tf.keras.Input(shape=(128,256,8))
    base_model = efn.EfficientNetB0(include_top=False, weights=None, input_shape=None)
    if pretrain:
        base_model.load_weights('/kaggle/input/tf-efficientnet-imagenet-weights/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5')

    # RESHAPE INPUT 128x256x8 => 512x512x3 MONOTONE IMAGE
    # KAGGLE SPECTROGRAMS
    x1 = [inp[:,:,:,i:i+1] for i in range(4)]
    x1 = tf.keras.layers.Concatenate(axis=1)(x1)
    # EEG SPECTROGRAMS
    x2 = [inp[:,:,:,i+4:i+5] for i in range(4)]
    x2 = tf.keras.layers.Concatenate(axis=1)(x2)
    # MAKE 512X512X3
    if USE_KAGGLE_SPECTROGRAMS & USE_EEG_SPECTROGRAMS:
        x = tf.keras.layers.Concatenate(axis=2)([x1,x2])
    elif USE_EEG_SPECTROGRAMS: x = x2
    else: x = x1
    x = tf.keras.layers.Concatenate(axis=3)([x,x,x])

    # OUTPUT
    x0 = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x0)
    x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)

    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=[x,x0])
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()

    model.compile(loss=loss, optimizer = opt)

    return model

In [ ]:
from sklearn.model_selection import KFold, GroupKFold

gkf = GroupKFold(n_splits=5)
for fold, (train_index, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):
    # LOAD WEIGHTS INTO GRAD CAM MODEL
    with strategy.scope():
        model = build_cam_model()
    model.load_weights(f'{LOAD_MODELS_FROM}EffNet_v{VER}_f{fold}.h5')
    layer_weights = model.layers[-1].get_weights()[0][:,0]
    break

print('Using fold 0 model and inferring fold 0 OOF (out of fold) samples...')

# Display Grad Cam
With grad cam, given a specific OOF (out of fold) train sample, we can view both a model's prediction and where it looked to make this prediction. In the plots below we display the image that was fed into our image model, in my popular starter notebook, there are 8 spectrograms that have been tiled into one 1 input image.

On the left we have the 4 Kaggle spectrograms where each is 10 minutes long. Each represents one of the 4 montages LL, RL, LP, RP. (Montages explained [here][1]) On the right, we have the 4 EEG spectrograms where each is 50 seconds long. The EEG spectrograms are made from the Magic Formula [here][2]. The spectrograms are each `128x256x1`, so the final concatenation is `512x512x1`.

![](https://raw.githubusercontent.com/cdeotte/Kaggle_Images/main/Feb-2024/key2.png)

Each plot below has 3 subplots. The middle and right subplot use the KEY above. The left subplot is just the Grad Cam image where larger values (more yellow) indicates more attention. The middle subplot is the contours of the Grad Cam's 10% largest values superimposed over the image that we fed into our model. The right subplot is also Grad Cam contour superimposed over image but we add an emboss filter to the image to make the details more visible to humans. For explanations about specific grad cam examples, see discussion [here][3]

[1]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467877
[2]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/469760
[3]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/472976

In [ ]:
import cv2

# HELPER FUNCTION
def mask2contour(mask, width=5):
    w = mask.shape[1]
    h = mask.shape[0]
    mask2 = np.concatenate([mask[:,width:],np.zeros((h,width))],axis=1)
    mask2 = np.logical_xor(mask,mask2)
    mask3 = np.concatenate([mask[width:,:],np.zeros((width,w))],axis=0)
    mask3 = np.logical_xor(mask,mask3)
    return np.logical_or(mask2,mask3)

clahe = cv2.createCLAHE(clipLimit=16.0, tileGridSize=(8,8))

In [ ]:
BATCH = 128

for ii,tt in enumerate(TARGETS):
    ttt = tt.split('_')[0].upper()

    print()
    print('#'*25)
    print('###',tt.upper())
    print('#'*25)

    # FIND TRAIN SAMPLES IN OOF (OUT OF FOLD) WITH TARGET >= 0.5
    IDX = train.loc[train.index.isin(valid_index) & (train[tt]>=0.5),TARGETS].index.values
    print(f'Found {len(IDX)} samples in fold zero OOF for {tt} with true>0.5')

    # INFER TRAIN SAMPLES WITH MODEL (SAVE PREDS AND ACTIVATIONS)
    valid_gen = DataGenerator(train.iloc[IDX[:128]], shuffle=False, batch_size=BATCH, mode='valid')
    p,xx = model.predict(valid_gen,verbose=0)
    #print(xx.shape)

    # DISPLAY GRAD CAM
    for x,y in valid_gen:
        ct = 0
        for i in range(BATCH):

            # FIND SAMPLES WITH PRED >= 0.5 FOR TARGET
            if i>=len(p): continue
            pred = p[i]
            if pred[ii]<0.5: continue

            # FORMAT PREDICTIONS AS STRING
            pred2 = ''; true2 = ''
            true = train.loc[IDX[i]][TARGETS].values
            for j,t in enumerate(TARGETS):
                n = t.split('_')[0]
                pred2 += f' {n}={pred[j]:0.3f}'
                true2 += f' {n}={true[j]:0.3f}'
            print()
            print('==> TRUE:',true2)
            print('==> PRED:',pred2)

            # PLOT GRAD CAM RESULTS
            plt.figure(figsize=(20,8))

            # PLOT GRAD CAM IMAGE (PLOT 1 OF 3)
            plt.subplot(1,3,1)
            img = np.sum(xx[i,] * layer_weights,axis=-1)
            img = cv2.resize(img,(512,512))
            plt.imshow(img[::-1,])
            plt.title(f'{ttt} - Grad Cam',size=14)

            # FIND GRAD CAM CONTOURS FOR AREAS OF INTEREST
            cut = np.percentile(img.flatten(), [90])[0]
            cntr = img.copy()
            cntr[cntr>=cut] = 100
            cntr[cntr<cut] = 0
            cntr = mask2contour(cntr)

            # PLOT EMBOSSED SPECTROGRAMS WITH GRADCAM CONTOURS (PLOT 3 OF 3)
            plt.subplot(1,3,3)
            x1 = [x[i,:,:,k:k+1] for k in range(4)] #KAGGLE-SPECS: LL RL LP RP
            x1 = np.concatenate(x1,axis=0)
            x2 = [x[i,:,:,k+4:k+5] for k in range(4)] #EEG-SPECS: LL LP RL RP
            x2 = np.concatenate(x2,axis=0)
            x3 = np.concatenate([x1,x2],axis=1)
            img = cv2.resize(x3,(512,512))
            img0 = img.copy()

            # EMBOSS IMAGE FOR IMAGE FEATURE VISIBILITY
            img = img[1:,1:] - img[:-1,:-1] #emboss
            img -= np.min(img)
            img /= np.max(img)
            img = (img*255).astype('uint8')
            img = cv2.GaussianBlur(img,(5,5),0)
            img = clahe.apply(img)
            mx = np.max(img)

            cntr2 = cntr[1:,1:]
            img[cntr2>0] = mx
            plt.imshow(img[::-1,])
            plt.title(f'{ttt} - Embossed Spectrogram with Grad Cam Contours',size=14)

            # PLOT SPECTROGRAMS WITH GRADCAM CONTOURS (PLOT 2 OF 3)
            plt.subplot(1,3,2)
            mx = np.max(img0)
            img0[cntr>0] = mx
            plt.imshow(img0[::-1,])
            plt.title(f'{ttt} - Spectrogram with Grad Cam Contours',size=14)

            plt.show()
            ct += 1
            if ct==8: break

        break